In [46]:
# ---------------------------------6.加载用于微调的数据-----------------------------------
# 在加载数据集时传入 tokenizer
ppath = r'G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\data\custom_tokenizer0318\vocabContourW0512.txt'
pretrained_model_path = r'G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\result\Cps30epoch40Len128LR1e-4Bt256warm1000_H512L8A8\checkpoint-63000' 
#Corpus&Lablel_Classified_Split257_XianYang_LineUnit_13万多线元用Split257相交     Corpus&Lablel_Classified_Split257_XianYang_LineUnit_8万多线元用代相交.txt
finTuneDataPath=r"G:\BT\data\3downStreamTask\SemanticSegmentation\3Courpus_LineUnit&TagID&Label\Corpus&Lablel_Classified_Split257_XianYang_LineUnit_13万多线元用Split257相交.txt"
outFineTuneModelPath=r"G:\BT\data\3downStreamTask\SemanticSegmentation\4FineTunedModel&Result\test"#"./dataTokenCls_SentencenSegmentation/result/fine_tuned_model"  
outResMetricPath=r"G:\BT\data\3downStreamTask\SemanticSegmentation\4FineTunedModel&Result"
batch_size=32
max_len=128#最大句子长度
num_labels = 2  # O: 普通token, SEP: 句子分割token
num_epochs = 30
save_eval_steps=100
save_total_limits=100
lr=5e-5
from transformers import BertForTokenClassification, Trainer, TrainingArguments,BertTokenizer,DataCollatorForTokenClassification
from torch.utils.data import Dataset, DataLoader
import torch,os
import matplotlib.pyplot as plt

# 禁用代理
os.environ["HTTP_PROXY"] = ""
os.environ["HTTPS_PROXY"] = ""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer(vocab_file=ppath, unk_token="[UNK]", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]", mask_token="[MASK]")
tokenizer.model_max_length = max_len
# tokenizer = BertTokenizer.from_pretrained(occDir)  # 或者BertTokenizer(occDir + '/vocab.txt')
# text = "urban function graph method"
# encoding = tokenizer(text, padding='max_length', truncation=True, max_length=max_len, return_tensors='pt')
# input_ids = encoding['input_ids']
# attention_mask = encoding['attention_mask']
# print("Input IDs:", input_ids)
# print("Attention Mask:", attention_mask)

class FineTuneDataset(Dataset):
    def __init__(self, file_path, tokenizer,max_len):
        self.samples = []
        # self.label_mapping = {0: "O", 1: "SEP"}  # 定义标签映射
        self.tokenizer = tokenizer
        self.max_len = max_len  # 添加max_len作为类的属性         
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                line = line.strip()  # "4 Results"
                tokens_and_labels = line.split()  # 按空格split
                tokens = tokens_and_labels[:-1]
                labels = [int(label) for label in tokens_and_labels[-1].split(",")]  # 按， split

                # 添加起始和结束标记否则会提示Using bos_token and eos_token, but it is not set yet. 是由于使用了预训练模型，而你的输入数据中没有设置起始标记（bos_token）和结束标记（eos_token）。这两个标记通常在训练时添加，但在推理时可能不需要。
                tokens = [self.tokenizer.cls_token] + tokens + [self.tokenizer.sep_token]  
                labels = [0] + labels + [0]  

                # 检查句子长度并截断  
                if len(tokens) > self.max_len:  
                    tokens = tokens[:self.max_len-1]  # 保留CLS但去掉超出长度的部分和SEP  
                    labels = labels[:self.max_len-1]  # 保留对应标签  
                    tokens.append(self.tokenizer.sep_token)  # 最后添加SEP  
                    labels.append(0)  # 添加对应SEP的标签  
  
                # 如果句子长度小于max_len，则进行填充  
                padding_length = self.max_len - len(tokens)  
                tokens += [self.tokenizer.pad_token] * padding_length  
                labels += [0] * padding_length  
  
                self.samples.append((tokens, labels))              

    def __len__(self):
        return len(self.samples)

    # def __getitem__(self, idx):
    #     return self.samples[idx]
    def __getitem__(self, idx):
        tokens, labels = self.samples[idx]
        return {
            "input_ids": torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens)),
            "attention_mask": torch.tensor([1] * len(tokens)),  # 根据需求自定义 attention_mask
            "labels": torch.tensor(labels),
        }

fine_tune_dataset = FineTuneDataset(finTuneDataPath, tokenizer,max_len)
fine_tune_dataloader = DataLoader(fine_tune_dataset, batch_size=batch_size, shuffle=False)

# 自定义的数据collator
pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorForTokenClassification(tokenizer, padding='max_length',max_length=max_len,label_pad_token_id=pad_token_id, pad_to_multiple_of=tokenizer.model_max_length)
# truncation=True, max_length=max_len, return_tensors='pt'
# ---------------------------------7.初始化用于微调的模型---------------------------------
# 加载预训练的模型
pretrained_model = BertForTokenClassification.from_pretrained(pretrained_model_path)

# 根据微调数据的标签数量调整分类器的输出维度（这里假设标签为O和SEP）

# 获取预训练模型的配置
config = pretrained_model.config

# 创建新的分类器，并设置正确的输入和输出维度
pretrained_model.classifier = torch.nn.Linear(config.hidden_size, num_labels)

# 确保微调模型输出层的权重被初始化
pretrained_model.classifier.weight.data.normal_(mean=0.0, std=config.initializer_range)
pretrained_model.classifier.bias.data.zero_()

# 设置微调 Trainer 参数
training_args_fine_tune = TrainingArguments(
    output_dir=outFineTuneModelPath,
    overwrite_output_dir=True,
    num_train_epochs=num_epochs,  # 调整微调轮数
    per_device_train_batch_size=batch_size,
    save_steps=save_eval_steps,
    eval_steps=save_eval_steps,
    save_total_limit=save_total_limits,
    report_to="tensorboard",
)

# 初始化微调 Trainer
trainer_fine_tune = Trainer(
    model=pretrained_model,
    args=training_args_fine_tune,
    train_dataset=fine_tune_dataset,
    data_collator=data_collator,  # 使用之前定义的 data_collator
)
# ---------------------------------8.开始微调，并保存微调后的模型--------------------------------
optimizer = torch.optim.AdamW(pretrained_model.parameters(), lr=lr)  # 初始化AdamW优化器
pretrained_model.to(device)

for epoch in range(num_epochs):
    pretrained_model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    predictions = []

    print(f"Epoch {epoch + 1}/{num_epochs}")
    for batch_idx, batch in enumerate(fine_tune_dataloader):
        # 获取数据
        inputs = {
            "input_ids": batch["input_ids"].to(torch.device("cuda")),
            "attention_mask": batch["attention_mask"].to(torch.device("cuda")),
            "labels": batch["labels"].to(torch.device("cuda")),
        }

        # 微调
        outputs = pretrained_model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()

        # 获取预测结果
        logits = outputs.logits
        predictions_batch = torch.argmax(logits, dim=num_labels)
        predictions.extend(predictions_batch.cpu().numpy())

        # 计算准确度
        labels = inputs["labels"]
        correct_predictions += (predictions_batch == labels).sum().item()
        total_samples += labels.numel()

        # 更新权重
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # # 保存当前 batch 的预测结果
        # batch_predictions_file = f"./dataTokenCls_SentencenSegmentation/result/batch_predictions_epoch_{epoch + 1}_batch_{batch_idx + 1}.txt"
        # with open(batch_predictions_file, "w", encoding="utf-8") as batch_file:
        #     for token_predictions in predictions_batch.cpu().numpy():
        #         token_predictions = [str(pred) for pred in token_predictions]
        #         batch_file.write(" ".join(token_predictions) + "\n")

    # 保存整个 epoch 的预测结果
    epoch_predictions_file = f"{outResMetricPath}epoch_predictions_epoch_{epoch + 1}.txt"
    with open(epoch_predictions_file, "w", encoding="utf-8") as epoch_file:
        for token_predictions in predictions:
            token_predictions = [str(pred) for pred in token_predictions]
            epoch_file.write(" ".join(token_predictions) + "\n")

    # 计算平均损失和准确度
    average_loss = total_loss / len(fine_tune_dataloader)
    accuracy = correct_predictions / total_samples
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}, Accuracy: {accuracy}")

    print("Saving model...")
    pretrained_model.save_pretrained(f"{outFineTuneModelPath}fine_tuned_model_epoch_{epoch + 1}")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\result\Cps30epoch40Len128LR1e-4Bt256warm1000_H512L8A8\checkpoint-63000 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/30
Epoch 1, Average Loss: 0.27340148850565865, Accuracy: 0.8851586138426626
Saving model...
Epoch 2/30
Epoch 2, Average Loss: 0.21866507757277714, Accuracy: 0.9063445537065052
Saving model...
Epoch 3/30
Epoch 3, Average Loss: 0.19325370261711733, Accuracy: 0.9207699035552194
Saving model...
Epoch 4/30
Epoch 4, Average Loss: 0.17014953077194236, Accuracy: 0.9295574886535553
Saving model...
Epoch 5/30
Epoch 5, Average Loss: 0.1587733184652669, Accuracy: 0.938557819591528
Saving model...
Epoch 6/30
Epoch 6, Average Loss: 0.12243428611240927, Accuracy: 0.9532018248865356
Saving model...
Epoch 7/30
Epoch 7, Average Loss: 0.09201033085229851, Accuracy: 0.963561365355522
Saving model...
Epoch 8/30
Epoch 8, Average Loss: 0.06669601585183825, Accuracy: 0.9746832450832073
Saving model...
Epoch 9/30
Epoch 9, Average Loss: 0.06336395209655166, Accuracy: 0.9757647031013615
Saving model...
Epoch 10/30
Epoch 10, Average Loss: 0.057694194106651206, Accuracy: 0.9775257658850227
Saving model...


In [51]:

# ---------------------------------9.（1）预测测试数据文件-----------也可预测微调数据，进行分析---------------------
import os,torch,itertools
# 禁用代理
os.environ["HTTP_PROXY"] = ""
os.environ["HTTPS_PROXY"] = ""
from transformers import BertTokenizer, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size=32
num_labels = 2  # O: 普通token, 1: 塬
max_len=128#最大句子长度
from sklearn.metrics import confusion_matrix
fine_tuned_model_path = r"G:\BT\data\3downStreamTask\SemanticSegmentation\4FineTunedModel&Result\test\fine_tuned_model_epoch_30"  # 替换成你保存微调后模型的路径
ppath = r'G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\data\custom_tokenizer0318\vocabContourW0512.txt'#词汇表
outResMetricPath=r'G:\BT\data\3downStreamTask\SemanticSegmentation\5TestResult'
testDatasetDirectory=r"G:\BT\data\3downStreamTask\SemanticSegmentation\3Courpus_LineUnit&TagID&Label"
testData_lst=[
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split255_BaoJi_LineUnit.txt'),
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split257_BaiYin_LineUnit.txt'),
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split257_PingLiang_LineUnit.txt'),
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split257_QingYang_LineUnit.txt'),
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split257_TongChuan_DelLineUnit.txt'),
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split257_WeiNan_DelLineUnit.txt'),
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split257_YunCheng_LineUnit.txt'),
    os.path.join(testDatasetDirectory,'Corpus&Lablel_Classified_Split257_XianYang_LineUnit.txt'),
]
# 定义待预测的数据集类
class TestDataset(Dataset):
    def __init__(self, file_path, tokenizer):
        self.samples = []
        # self.label_mapping = {0: "O", 1: "SEP"}  # 定义标签映射
        self.tokenizer = tokenizer
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                line = line.strip()  # "4 Results"
                tokens_and_labels = line.split()  # 按空格split
                tokens = tokens_and_labels[:-1]
                labels = [int(label) for label in tokens_and_labels[-1].split(",")]  # 按， split

                # 添加起始和结束标记否则会提示Using bos_token and eos_token, but it is not set yet. 是由于使用了预训练模型，而你的输入数据中没有设置起始标记（bos_token）和结束标记（eos_token）。这两个标记通常在训练时添加，但在推理时可能不需要。
                tokens = [self.tokenizer.cls_token] + tokens + [self.tokenizer.sep_token]
                labels = [0] + labels + [0]

                self.samples.append((tokens, labels))
                # print((tokens, labels))
        # 手动填充：
        for i in range(len(self.samples)):
            tokens, labels = self.samples[i]
            padding_length = max_len - len(tokens)
            tokens += [self.tokenizer.pad_token] * padding_length
            labels += [0] * padding_length
            self.samples[i] = (tokens, labels)                  

    def __len__(self):
        return len(self.samples)

    # def __getitem__(self, idx):
    #     return self.samples[idx]
    def __getitem__(self, idx):
        tokens, labels = self.samples[idx]
        return {
            "input_ids": torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens)),
            "attention_mask": torch.tensor([1] * len(tokens)),  # 根据需求自定义 attention_mask
            "labels": torch.tensor(labels),
        }

# 绘制混淆矩阵 显示百分比或数值
def plot_confusion_matrix_percent(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues,confusionMatrixPngPath='./confusionMatrixPng.png'):
    """
    - cm : 计算出的混淆矩阵的值
    - classes : 混淆矩阵中每一行每一列对应的列
    - normalize : True:显示百分比, False:显示个数
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print("混淆矩阵：")
        np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
        # print(cm)
    else:
        print()
        # print('混淆矩阵：')
        # print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    # matplotlib版本问题，如果不加下面这行代码，则绘制的混淆矩阵上下只能显示一半，有的版本的matplotlib不需要下面的代码，分别试一下即可
    plt.ylim(len(classes) - 0.5, -0.5)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # plt.show()
    plt.savefig(confusionMatrixPngPath)
    plt.close()
    
# 1. 加载微调后的模型
fine_tuned_model = BertForTokenClassification.from_pretrained(fine_tuned_model_path,output_hidden_states=True).to(device)
# 2. 加载微调时使用的tokenizer
tokenizer = BertTokenizer(vocab_file=ppath, unk_token="[UNK]", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]", mask_token="[MASK]")
tokenizer.model_max_length = max_len
for testDatasetPath in testData_lst:
    outResultFileName="PreLabel"+os.path.basename(testDatasetPath)
    predictions_file=os.path.join(outResMetricPath, outResultFileName)
    if os.path.exists(predictions_file): 
        # 如果文件存在，则删除它  
        try:  
            os.remove(predictions_file)  
            print(f"文件 {predictions_file} 已成功删除。")  
        except OSError as e:  
            print(f"删除文件 {predictions_file} 时出错: {e.strerror}") 

    # 3. 加载测试数据集
    test_dataset = TestDataset(testDatasetPath, tokenizer)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)#, collate_fn=lambda batch: collate_fn(batch, tokenizer))

    # 4. 使用微调后的模型进行预测
    fine_tuned_model.eval()
    total_samples,total_loss,correct_predictions = 0,0,0
    average_loss,accuracy=0,0
    predictions,labels,predictions2 = [],[],[]
    with torch.no_grad():
        for batch in test_dataloader:
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device),
                "labels": batch["labels"].to(device),
            }
            outputs = fine_tuned_model(**inputs)
            # predicted_labels = torch.argmax(outputs.logits, dim=2).tolist()
            # predictions.extend(predicted_labels)
        
            loss = outputs.loss
            total_loss += loss.item()

            # 获取预测结果
            logits = outputs.logits
            predictions_batch = torch.argmax(logits, dim=num_labels)

            # 计算准确度
            labels_batch = inputs["labels"]
            correct_predictions += (predictions_batch == labels_batch).sum().item()
            total_samples += labels_batch.numel()  
            
            predictions_batch  =predictions_batch.cpu().numpy().tolist()   
            predictions2+=predictions_batch
            labels_batch=labels_batch.cpu().numpy().tolist()
            for item_pre in predictions_batch:
                predictions+=item_pre
            for item_label in labels_batch:
                labels+=item_label           
                 
        # 计算平均损失和准确度
        average_loss =np.round(total_loss / len(test_dataloader),decimals=4)
        accuracy = np.round(correct_predictions / total_samples,decimals=4)
        print(outResultFileName+f"  Average Loss: {average_loss}, Accuracy: {accuracy}")       
    # 5. 输出打印预测结果

    # for i, pred in enumerate(predictions):
    #     tokens = tokenizer.convert_ids_to_tokens(test_dataset[i]["input_ids"])
    #     print(f"Sample {i + 1} - Tokens: {tokens}")
    #     print(f"Sample {i + 1} - Predicted Labels: {pred}")
    #     print()
    with open(predictions_file, "w", encoding="utf-8") as file:
        for token_predictions in predictions2:
            token_predictions = [str(pred) for pred in token_predictions]
            file.write(" ".join(token_predictions) + "\n")
    #混淆矩阵（测试数据的预测结果 ）
    attack_types = ['other','Loess tablelannd']
    cm_train= confusion_matrix(labels,predictions)
    plot_confusion_matrix_percent(cm_train, classes=attack_types, normalize=False, title=f"  Average Loss {average_loss}, Accuracy {accuracy}",confusionMatrixPngPath=os.path.join(outResMetricPath,f"  Average Loss {average_loss}, Accuracy {accuracy}"+outResultFileName+".png"))
             
print('完成测试')

文件 G:\BT\data\3downStreamTask\SemanticSegmentation\5TestResult\PreLabelCorpus&Lablel_Classified_Split255_BaoJi_LineUnit.txt 已成功删除。
PreLabelCorpus&Lablel_Classified_Split255_BaoJi_LineUnit.txt  Average Loss: 0.1346, Accuracy: 0.98

文件 G:\BT\data\3downStreamTask\SemanticSegmentation\5TestResult\PreLabelCorpus&Lablel_Classified_Split257_BaiYin_LineUnit.txt 已成功删除。
PreLabelCorpus&Lablel_Classified_Split257_BaiYin_LineUnit.txt  Average Loss: 0.7429, Accuracy: 0.8906

文件 G:\BT\data\3downStreamTask\SemanticSegmentation\5TestResult\PreLabelCorpus&Lablel_Classified_Split257_PingLiang_LineUnit.txt 已成功删除。
PreLabelCorpus&Lablel_Classified_Split257_PingLiang_LineUnit.txt  Average Loss: 0.2494, Accuracy: 0.963

文件 G:\BT\data\3downStreamTask\SemanticSegmentation\5TestResult\PreLabelCorpus&Lablel_Classified_Split257_QingYang_LineUnit.txt 已成功删除。
PreLabelCorpus&Lablel_Classified_Split257_QingYang_LineUnit.txt  Average Loss: 0.2334, Accuracy: 0.9681

文件 G:\BT\data\3downStreamTask\SemanticSegmentation\5Tes